<a href="https://colab.research.google.com/github/mostafa-ja/Video_Dubber/blob/main/TTS_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai-whisper yt-dlp pysrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.3 MB/s

In [ ]:
import yt_dlp
import whisper
import os

# extract audio

In [4]:
youtube_url = "https://www.youtube.com/watch?v=ISToBIkSNbM"
output_path="audio"

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': output_path,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    'quiet': False
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=ISToBIkSNbM
[youtube] ISToBIkSNbM: Downloading webpage
[youtube] ISToBIkSNbM: Downloading tv client config
[youtube] ISToBIkSNbM: Downloading tv player API JSON
[youtube] ISToBIkSNbM: Downloading ios player API JSON
[youtube] ISToBIkSNbM: Downloading m3u8 information
[info] ISToBIkSNbM: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of   16.53MiB in 00:00:01 at 15.63MiB/s  
[ExtractAudio] Destination: audio.wav
Deleting original file audio (pass -k to keep)


# openai-whisper

Whisper's transcription function offers several optional parameters that you can customize based on your needs.

Here’s a breakdown of the most commonly used parameters in:

```python
model.transcribe(audio_path, **options)
```

---

## 📘 Common Parameters in `whisper.transcribe`

| Parameter                     | Type                     | Description                                                                                    |
| ----------------------------- | ------------------------ | ---------------------------------------------------------------------------------------------- |
| `audio`                       | `str` or `np.ndarray`    | Path to audio file or raw waveform (can be `.mp3`, `.wav`, `.m4a`, etc.).                      |
| `language`                    | `str`                    | Language code (e.g. `"en"`, `"de"`, `"hi"`). If not specified, Whisper will try to detect it.  |
| `task`                        | `str`                    | `"transcribe"` (default) or `"translate"` (translates to English).                             |
| `verbose`                     | `bool`                   | If `True`, prints progress and debug info. Default is `False`.                                 |
| `best_of`                     | `int`                    | Number of candidates evaluated to pick best (higher improves quality, slower). Default is `5`. |
| `beam_size`                   | `int`                    | Beam search width. If set, enables beam search decoding (better but slower).                   |
| `temperature`                 | `float` or `list[float]` | Sampling temperature. Higher values (e.g. 1.0) increase randomness. List allows fallback.      |
| `fp16`                        | `bool`                   | Use float16 precision (default `True` if GPU is available). Disable for CPU.                   |
| `initial_prompt`              | `str`                    | A string prompt to prime the model before transcribing (useful for domain-specific context).   |
| `condition_on_previous_text`  | `bool`                   | If `True`, uses previous text as context for next segment. Helps with coherence.               |
| `compression_ratio_threshold` | `float`                  | Filter threshold for flagging segments with too much repetition.                               |
| `logprob_threshold`           | `float`                  | If avg log-prob < threshold, segment is treated as low-confidence.                             |
| `no_speech_threshold`         | `float`                  | Probability threshold to detect silence (and skip segment).                                    |
| `suppress_tokens`             | `List[int]` or `str`     | Tokens to suppress from decoding. `"-1"` disables suppression.                                 |
| `max_initial_timestamp`       | `float`                  | Maximum timestamp allowed for the first segment (useful to trim start).                        |

---

## 🧠 Example Usage with Parameters

```python
result = model.transcribe(
    "audio.mp3",
    language="en",
    task="transcribe",
    verbose=True,
    beam_size=5,
    temperature=[0.0, 0.2, 0.4],
    fp16=False,
    initial_prompt="This is a lecture about machine learning."
)
```

---

## 🛠 When to Use What

* ✅ Use `beam_size` and `temperature=[0.0, 0.2, 0.4]` for **better transcription accuracy**.
* 🧪 Use `initial_prompt` when Whisper has trouble staying on-topic (e.g., with specialized language).
* ❄️ Use `fp16=False` **if you're on CPU** and get errors or slowdown with default settings.

---




## 🧭 1. `beam_size`: Beam Search Decoding

### 🔹 What it does:

* Beam search is a **smarter decoding strategy** than simple greedy decoding.
* It keeps track of **multiple possible transcriptions (beams)** at each step and only continues the most promising ones.
* Whisper evaluates each possible path using its internal probability model and picks the one with the **highest overall likelihood**.

### 🔹 Values:

* `beam_size=1` → Greedy decoding (fast but can miss better alternatives).
* `beam_size=5` or `10` → Better results, especially on hard audio, at the cost of speed.

### 🔹 Trade-off:

* **Higher `beam_size` = Better accuracy**, but **slower** transcription.
* Useful when audio is noisy, complex, or multilingual.

### ✅ Example:

```python
result = model.transcribe("audio.mp3", beam_size=5)
```

---

## 🔥 2. `temperature`: Sampling Temperature

### 🔹 What it does:

* Controls the **randomness or creativity** in choosing the next word/token.
* **Low temperature (0.0–0.3)** = very deterministic.
* **High temperature (0.8–1.0)** = more diverse and random outputs.

### 🔹 Values:

* `temperature=0.0` → Always picks the most likely word. Deterministic.
* `temperature=1.0` → Picks from a wider range of possibilities, even if less likely.

### 🔹 Can use a **list**:

Whisper will try each temperature in the list in order until a high-confidence result is found (good fallback behavior):

```python
temperature=[0.0, 0.2, 0.4]
```

### ✅ Example:

```python
result = model.transcribe("audio.mp3", temperature=0.2)
```

---

## 🧠 When to Use Each

| Scenario                                                        | Recommended `beam_size` | Recommended `temperature` |
| --------------------------------------------------------------- | ----------------------- | ------------------------- |
| Clean audio, fast output                                        | 1                       | 0.0                       |
| Noisy audio                                                     | 5–10                    | 0.0–0.4                   |
| Creative or variable transcription (e.g. for subtitles, poetry) | 3–5                     | 0.7–1.0                   |
| Use fallback mechanism                                          | 5                       | `[0.0, 0.2, 0.4]`         |

---


In [3]:
model = whisper.load_model("large")
result = model.transcribe(
    "audio.wav",
    language="en",
    task="transcribe",
    verbose=True,
    beam_size=5,
)

[00:00.040 --> 00:11.760]  Good morning from the town of Vjetka here in Eastern Belarus. About two months ago I took a
[00:11.760 --> 00:16.020]  journey into the irradiated forests that surround this town and there in a small hamlet in the
[00:16.020 --> 00:21.000]  middle of nowhere I met an elderly mother and her son. They invited me into their cottage and they
[00:21.000 --> 00:25.200]  shared the contents of their fridge with me and so I've come back to the country to return the
[00:25.200 --> 00:31.320]  favor. So I'm now going to go to a supermarket, buy some essentials, buy some food, buy some vodka
[00:31.320 --> 00:36.180]  and I'm going to travel back there into the zone to find them and return the favor. Alright,
[00:36.180 --> 00:39.120]  join me on a journey into the zone. Let's do it!
[00:55.200 --> 01:12.240]  As we head north from Chechersk, I'd like to show you something on this road,
[01:12.240 --> 01:16.860]  this highway, and that is that every few kilometers you c

In [4]:
from whisper.utils import get_writer

audio = "/content/audio.wav"
output_directory = "./"

# Save as an SRT file
srt_writer = get_writer("srt", output_directory)
srt_writer(result, audio)



# translation

In [ ]:
!pip install --upgrade gemini-srt-translator

In [1]:
import gemini_srt_translator as gst

gst.gemini_api_key = ""
gst.target_language = "Persian"
gst.input_file = "Chernobyl.srt"

gst.translate()

Starting translation of 187 lines...

Translating: |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% (0/187) gemini-2.5-flash-preview-05-20 | Sending batch ↑↑↑

Translating: |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% (0/187) gemini-2.5-flash-preview-05-20

Validating token size...
Translating: |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% (0/187) gemini-2.5-flash-preview-05-20 | Sending batch ↑↑↑

Validating token size...
Token size validated. Translating...
Translating: |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% (0/187) gemini-2.5-flash-preview-05-20 | Thinking —

Validating token size...
Token size validated. Translating...
Translating: |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% (0/187) gemini-2.5-flash-preview-05-20 | Thinking \

Validating token size...
Token size validated. Translating...
Translating: |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% (0/187) gemini-2.5-flash-preview-05-20 | Thinking |

Validating token size...
Token size validated. Translating...
Translating: |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% (0/187) gemini-2.5-flash-pr

# Text to speeach

In [140]:
import base64
import mimetypes
import os
import re
import struct
from google import genai
from google.genai import types


def save_binary_file(file_name, data):
    f = open(file_name, "wb")
    f.write(data)
    f.close()
    print(f"File saved to to: {file_name}")

def convert_to_wav(audio_data: bytes, mime_type: str) -> bytes:
    """Generates a WAV file header for the given audio data and parameters.

    Args:
        audio_data: The raw audio data as a bytes object.
        mime_type: Mime type of the audio data.

    Returns:
        A bytes object representing the WAV file header.
    """
    parameters = parse_audio_mime_type(mime_type)
    bits_per_sample = parameters["bits_per_sample"]
    sample_rate = parameters["rate"]
    num_channels = 1
    data_size = len(audio_data)
    bytes_per_sample = bits_per_sample // 8
    block_align = num_channels * bytes_per_sample
    byte_rate = sample_rate * block_align
    chunk_size = 36 + data_size  # 36 bytes for header fields before data chunk size

    # http://soundfile.sapp.org/doc/WaveFormat/

    header = struct.pack(
        "<4sI4s4sIHHIIHH4sI",
        b"RIFF",          # ChunkID
        chunk_size,       # ChunkSize (total file size - 8 bytes)
        b"WAVE",          # Format
        b"fmt ",          # Subchunk1ID
        16,               # Subchunk1Size (16 for PCM)
        1,                # AudioFormat (1 for PCM)
        num_channels,     # NumChannels
        sample_rate,      # SampleRate
        byte_rate,        # ByteRate
        block_align,      # BlockAlign
        bits_per_sample,  # BitsPerSample
        b"data",          # Subchunk2ID
        data_size         # Subchunk2Size (size of audio data)
    )
    return header + audio_data

def parse_audio_mime_type(mime_type: str) -> dict[str, int | None]:
    """Parses bits per sample and rate from an audio MIME type string.

    Assumes bits per sample is encoded like "L16" and rate as "rate=xxxxx".

    Args:
        mime_type: The audio MIME type string (e.g., "audio/L16;rate=24000").

    Returns:
        A dictionary with "bits_per_sample" and "rate" keys. Values will be
        integers if found, otherwise None.
    """
    bits_per_sample = 16
    rate = 24000

    # Extract rate from parameters
    parts = mime_type.split(";")
    for param in parts: # Skip the main type part
        param = param.strip()
        if param.lower().startswith("rate="):
            try:
                rate_str = param.split("=", 1)[1]
                rate = int(rate_str)
            except (ValueError, IndexError):
                # Handle cases like "rate=" with no value or non-integer value
                pass # Keep rate as default
        elif param.startswith("audio/L"):
            try:
                bits_per_sample = int(param.split("L", 1)[1])
            except (ValueError, IndexError):
                pass # Keep bits_per_sample as default if conversion fails

    return {"bits_per_sample": bits_per_sample, "rate": rate}

def generate(text_file_path):
    # Step 1: Read text from file
    with open(text_file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Step 2: Initialize Gemini client
    client = genai.Client(api_key="")
    model = "gemini-2.5-flash-preview-tts"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=text),
            ],
        ),
    ]



    generate_content_config = types.GenerateContentConfig(
        temperature=0.8,
        response_modalities=["audio"],
        speech_config=types.SpeechConfig(
            voice_config=types.VoiceConfig(
                prebuilt_voice_config=types.PrebuiltVoiceConfig(
                    voice_name="Charon"
                )
            )
        ),
    )

    file_index = 0
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if (
            chunk.candidates is None
            or chunk.candidates[0].content is None
            or chunk.candidates[0].content.parts is None
        ):
            continue

        part = chunk.candidates[0].content.parts[0]
        if part.inline_data and part.inline_data.data:
            inline_data = part.inline_data
            data_buffer = inline_data.data
            file_extension = mimetypes.guess_extension(inline_data.mime_type)
            if file_extension is None:
                file_extension = ".wav"
                data_buffer = convert_to_wav(inline_data.data, inline_data.mime_type)

            file_name = f"output_audio_{file_index}{file_extension}"
            file_index += 1
            save_binary_file(file_name, data_buffer)
        else:
            print(chunk.text)


## by chunks

In [139]:
def srt_to_text_split(input_file, output_prefix):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    output_lines = []
    buffer = []

    for line in lines:
        line = line.strip()
        if line.isdigit():
            continue  # skip subtitle number
        elif '-->' in line:
            continue  # skip timestamp
        elif line == '':
            if buffer:
                output_lines.append(' '.join(buffer))
                buffer = []
        else:
            buffer.append(line)

    if buffer:
        output_lines.append(' '.join(buffer))

    # Split into chunks of 50 lines and write to separate files
    chunk_size = 50
    for i in range(0, len(output_lines), chunk_size):
        chunk = output_lines[i:i + chunk_size]
        file_number = (i // chunk_size) + 1
        output_file = f"{output_prefix}_{file_number}.txt"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write('این متن را سریع بخوان و درانتهای هر خط حتما  به اندازه یک ثانیه مکث کن و سپس ادامه بده : ' + '\n\n\n')
            for line in chunk:
                cleaned_line = line.replace('.', '').replace('،', '') + '.'
                f.write(cleaned_line + '\n'+'.'+'\n'+'.'+'\n')

# Example usage:
srt_to_text_split('translated.srt', 'lines')


In [142]:
import wave
import os

def combine_wav_files(output_filename, input_filenames):
    # Open the first file to get audio parameters
    with wave.open(input_filenames[0], 'rb') as wav:
        print(wav.getparams())
        params = wav.getparams()
        frames = wav.readframes(wav.getnframes())

    # Open output file and set parameters
    with wave.open(output_filename, 'wb') as output:
        output.setparams(params)
        output.writeframes(frames)

        # Append rest of the files
        for filename in input_filenames[1:]:
            with wave.open(filename, 'rb') as wav:
                print(wav.getparams())
                #if wav.getparams() != params:
                    #raise ValueError(f"Audio format mismatch in file: {filename}")
                frames = wav.readframes(wav.getnframes())
                output.writeframes(frames)

    print(f"Combined audio saved as: {output_filename}")

# Example usage
# Assumes files are named chunk_1.wav, chunk_2.wav, ..., chunk_n.wav
input_files = [f"chunk_{i}.wav" for i in range(1, 6)]  # Update the range if needed
combine_wav_files("combined_output.wav", input_files)


_wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=5592263, comptype='NONE', compname='not compressed')
_wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=5589383, comptype='NONE', compname='not compressed')
_wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=3872903, comptype='NONE', compname='not compressed')
_wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=2670983, comptype='NONE', compname='not compressed')
_wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=2022983, comptype='NONE', compname='not compressed')
Combined audio saved as: combined_output.wav


In [149]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os


# Load the audio file
sound = AudioSegment.from_file("combined_output.wav", format="wav")  # or mp3

# Split audio where silence is longer than 1000ms (1s) and silence threshold is -40 dBFS
chunks = split_on_silence(
    sound,
    min_silence_len=400,  # silence duration in ms
    silence_thresh=sound.dBFS - 10,  # silence threshold
    keep_silence=20  # optional: keep a bit of silence at the edges
)


# Create the 'data' folder if it doesn't exist
os.makedirs("data", exist_ok=True)

# Export chunks to the 'data' folder
for i, chunk in enumerate(chunks):
    chunk.export(f"data/chunk_{i+1}.wav", format="wav")

In [150]:
folder_path = "data"
num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
print(f"Number of files in '{folder_path}': {num_files}")


Number of files in 'data': 242


In [148]:
import shutil

folder_path = "data"

# WARNING: This will permanently delete the folder and everything inside it
shutil.rmtree(folder_path)
print(f"Deleted folder '{folder_path}' and all its contents.")


Deleted folder 'data' and all its contents.


In [151]:
import subprocess
from pydub import AudioSegment
import pysrt
import os

def stretch_audio_to_fit(input_wav, output_wav, target_duration_ms):
    """
    Use ffmpeg to change the speed of input_wav so its duration matches target_duration_ms
    """
    original = AudioSegment.from_wav(input_wav)
    original_duration = len(original)

    speed_factor = original_duration / target_duration_ms
    speed_factor = max(0.5, min(speed_factor, 2.0))  # clamp for quality

    if abs(speed_factor - 1.0) < 0.01:
        # No need to change speed
        os.rename(input_wav, output_wav)
        return

    # FFmpeg command to adjust speed (tempo)
    # "atempo" supports values between 0.5 and 2.0
    subprocess.call([
        "ffmpeg", "-y", "-i", input_wav,
        "-filter:a", f"atempo={speed_factor}",
        output_wav
    ])



In [153]:
import pysrt
from pydub import AudioSegment
import os

# Load subtitles
subs = pysrt.open('translated.srt', encoding='utf-8')

# Get total duration: the end time of the last subtitle
last_end = subs[-1].end
total_duration = (
    last_end.hours * 3600 + last_end.minutes * 60 + last_end.seconds
) * 1000 + last_end.milliseconds

# Start with silent base
combined = AudioSegment.silent(duration=total_duration)

for i, sub in enumerate(subs):
    # Convert SRT times to milliseconds
    start_time = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
    end_time = (sub.end.hours * 3600 + sub.end.minutes * 60 + sub.end.seconds) * 1000 + sub.end.milliseconds
    time_window = end_time - start_time

    # Load corresponding audio chunk
    chunk_path = f"data/chunk_{i+1}.wav"
    if not os.path.exists(chunk_path):
        print(f"❌ Warning: {chunk_path} not found, skipping.")
        continue

    chunk = AudioSegment.from_wav(chunk_path)
    duration_chunk = len(chunk)

    if duration_chunk > time_window:
        print(f"❌ Warning: {chunk_path} is too long, {(duration_chunk - time_window)/1000} secound")
        temp_stretched = f"chunk_stretched_{i+1}.wav"
        stretch_audio_to_fit(chunk_path, temp_stretched, time_window)
        chunk = AudioSegment.from_wav(temp_stretched)
    else:
        chunk = AudioSegment.from_wav(chunk_path)


    # Overlay the chunk at the correct position
    combined = combined.overlay(chunk, position=start_time)

    # Get duration in milliseconds

    # Optionally delete chunk
    #os.remove(temp_file)

# Export the final audio
combined.export("combined_audio.wav", format="wav")
print("✅ Combined audio saved as 'combined_audio.wav'")

❌ Warning: data/chunk_2.wav is too long, 2.782 secound
❌ Warning: data/chunk_3.wav is too long, 1.131 secound
❌ Warning: data/chunk_4.wav is too long, 9.989 secound
❌ Warning: data/chunk_7.wav is too long, 2.288 secound
❌ Warning: data/chunk_13.wav is too long, 1.201 secound
❌ Warning: data/chunk_16.wav is too long, 1.707 secound
❌ Warning: data/chunk_18.wav is too long, 11.875 secound
❌ Warning: data/chunk_21.wav is too long, 6.53 secound
❌ Warning: data/chunk_24.wav is too long, 3.008 secound
❌ Warning: data/chunk_25.wav is too long, 1.667 secound
❌ Warning: data/chunk_27.wav is too long, 8.056 secound
❌ Warning: data/chunk_29.wav is too long, 0.56 secound
❌ Warning: data/chunk_30.wav is too long, 14.457 secound
❌ Warning: data/chunk_31.wav is too long, 19.441 secound
❌ Warning: data/chunk_32.wav is too long, 4.775 secound
❌ Warning: data/chunk_37.wav is too long, 0.767 secound
❌ Warning: data/chunk_43.wav is too long, 4.546 secound
❌ Warning: data/chunk_45.wav is too long, 0.166 sec

## line by line

In [ ]:
api_keys = [

]

In [ ]:
# To Resume After Timeout or Crash
# It will load combined_audio_partial.wav and continue from that point

with open('progress.txt', 'w') as f:
    f.write('34')  # replace with last completed index

In [ ]:
import pysrt
from pydub import AudioSegment
import os

# Load subtitles
subs = pysrt.open('translated.srt', encoding='utf-8')

# Get total duration: the end time of the last subtitle
last_end = subs[-1].end
total_duration = (
    last_end.hours * 3600 + last_end.minutes * 60 + last_end.seconds
) * 1000 + last_end.milliseconds

# Load from partial combined audio if exists
partial_audio_path = 'combined_audio_partial.wav'
if os.path.exists(partial_audio_path):
    combined = AudioSegment.from_wav(partial_audio_path)
    print("🔄 Loaded partial audio from previous run.")
else:
    combined = AudioSegment.silent(duration=total_duration)
    print("🎬 Starting new combined audio.")

# Read last completed index
checkpoint_file = 'progress.txt'
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        last_index = int(f.read().strip())
else:
    last_index = -1

print(f"🔁 Resuming from subtitle index {last_index + 1}")

for i, sub in enumerate(subs):
    if i <= last_index:
        continue

    try:
        # Convert SRT times to milliseconds
        start_time = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
        end_time = (sub.end.hours * 3600 + sub.end.minutes * 60 + sub.end.seconds) * 1000 + sub.end.milliseconds
        time_window = end_time - start_time

        input_text = "read the below line a bit faster: \n" + sub.text
        print(f"🎤 Generating audio for subtitle {i}: {input_text}")
        n = i % len(api_keys)
        generate(input_text, api_keys[n])
        time.sleep(10)  # wait 2 seconds

        chunk_path = f"data/chunk_{i+1}.wav"
        os.rename('output_audio_0.wav', chunk_path)

        if not os.path.exists(chunk_path):
            print(f"❌ Warning: {chunk_path} not found, skipping.")
            continue

        chunk = AudioSegment.from_wav(chunk_path)
        duration_chunk = len(chunk)

        if duration_chunk > time_window:
            print(f"⚠️ Warning: {chunk_path} is too long by {(duration_chunk - time_window)/1000:.2f} seconds")
            temp_stretched = f"chunk_stretched_{i+1}.wav"
            stretch_audio_to_fit(chunk_path, temp_stretched, time_window)
            chunk = AudioSegment.from_wav(temp_stretched)

        # Overlay the chunk at the correct position
        combined = combined.overlay(chunk, position=start_time)

        # ✅ Save checkpoint
        with open(checkpoint_file, 'w') as f:
            f.write(str(i))

        # ✅ Save current audio progress
        combined.export(partial_audio_path, format="wav")

    except Exception as e:
        print(f"❌ Error at subtitle {i}: {e}")
        print(f"💥 Last completed index: {i-1}")
        print(f"➡️ To resume from here, run this in a new cell:")
        print(f"with open('progress.txt', 'w') as f:\n    f.write('{i-1}')")
        break

# ✅ Final export (optional overwrite of full audio)
combined.export("combined_audio.wav", format="wav")
print("✅ Final combined audio saved as 'combined_audio.wav'")


🔄 Loaded partial audio from previous run.
🔁 Resuming from subtitle index 44
🎤 Generating audio for subtitle 44: read the below line a bit faster: 
‫تخریب شده‌اند.‬
❌ Error at subtitle 44: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerModelPerMinute-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-pro-tts', 'location': 'global'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-pr

# copy to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil

# dubbed audio
source_path = "/content/combined_audio.wav"
destination_path = "/content/drive/My Drive/combined_audio.wav"
shutil.copy(source_path, destination_path)

# translated srt
source_path = "/content/translated.srt"
destination_path = "/content/drive/My Drive/translated.srt"
shutil.copy(source_path, destination_path)


print("✅ Files copied to Google Drive.")

✅ File copied to Google Drive.
